## Simulations to verify network regularization

* Take genes from TCGA pancancer dataset with top $k$ coefficients and connect them in a network (unweighted or randomly weighted)
* Baseline: network from $k$ random genes in dataset
* Could subset data features to ($k$ top coefficients + $k$ random genes)

TODO: way to vary ratio of correlated/random features

In [1]:
import os
import sys; sys.path.append('..')
import numpy as np
import pandas as pd
import networkx as nx

import config as cfg
np.random.seed(cfg.default_seed)

In [2]:
tcga_train_df = pd.read_csv(cfg.rnaseq_train, index_col=0, sep='\t')
tcga_train_df.head()

,1,10,100,1000,10000,10001,10002,10003,100037417,10004,...,9987,9988,9989,999,9990,9991,9992,9993,9994,9997
sample_id,,,,,,,,,,,,,,,,,,,,,
TCGA-LL-A73Z-01,202.0,28.5,329.0,84.5,492.0,448.0,4.590,14.70,337.0,129.0,...,3430.0,717.0,1800.0,6360.0,299.0,2310.0,10.60,3190.0,337.0,892.0
TCGA-55-8207-01,77.5,22.5,74.5,13.1,784.0,333.0,2.540,176.00,153.0,68.3,...,6050.0,923.0,2490.0,11300.0,1150.0,4030.0,9.08,2890.0,316.0,301.0
TCGA-FF-A7CR-01,152.0,0.0,3020.0,26.6,486.0,497.0,0.000,8.47,348.0,91.6,...,4930.0,897.0,861.0,39.7,464.0,3320.0,0.00,1330.0,606.0,558.0
TCGA-BK-A13C-11,80.5,40.0,70.6,284.0,2420.0,325.0,1.200,91.40,231.0,241.0,...,3890.0,737.0,1410.0,10.9,1120.0,1990.0,5.24,3090.0,673.0,263.0
TCGA-EB-A6L9-06,319.0,0.0,422.0,184.0,423.0,392.0,0.945,2.36,585.0,143.0,...,1930.0,328.0,1340.0,7010.0,450.0,563.0,10.90,3780.0,37.3,1120.0


In [18]:
raw_results = os.path.join(cfg.results_dir, 
                           'canonical_pathways',
                           'mutation',
                           'TP53')
                           
raw_coefficients_df = pd.read_csv(os.path.join(raw_results,
                                               'TP53_raw_coefficients.tsv.gz'),
                                  sep='\t')
raw_coefficients_df = raw_coefficients_df.loc[raw_coefficients_df['signal'] == 'signal']

# drop non-gene features, including them causes issues later on 
raw_coefficients_df['feature'] = pd.to_numeric(raw_coefficients_df['feature'], errors='coerce')
raw_coefficients_df = raw_coefficients_df.dropna(subset=['feature'])
raw_coefficients_df.feature = raw_coefficients_df.feature.astype('int').astype('str')

raw_coefficients_df.iloc[100:110]

,feature,weight,abs,signal,z_dim,seed,algorithm,gene
101,9778,-0.017756,0.017756,signal,8000,42,raw,TP53
102,146691,-0.017704,0.017704,signal,8000,42,raw,TP53
103,689,-0.017069,0.017069,signal,8000,42,raw,TP53
104,51287,0.016733,0.016733,signal,8000,42,raw,TP53
105,53832,-0.016591,0.016591,signal,8000,42,raw,TP53
106,23476,0.016112,0.016112,signal,8000,42,raw,TP53
107,51507,0.015991,0.015991,signal,8000,42,raw,TP53
108,1635,-0.015967,0.015967,signal,8000,42,raw,TP53
109,55160,0.015684,0.015684,signal,8000,42,raw,TP53
110,84913,-0.015671,0.015671,signal,8000,42,raw,TP53


In [27]:
n_weights = 10
top_feats = raw_coefficients_df.iloc[:n_weights, :].feature.values
top_weights = raw_coefficients_df.iloc[:n_weights, :].weight.values
random_df = raw_coefficients_df[raw_coefficients_df.weight == 0.0]
random_feats = np.random.choice(random_df.feature.values,
                                size=10*top_feats.shape[0],
                                replace=False)
random_weights = raw_coefficients_df[raw_coefficients_df.feature.isin(random_feats)].weight.values
assert np.all(top_weights != 0.0)
assert np.all(random_weights == 0.0)

In [28]:
import itertools as it

# unweighted network from top features (and random features)
n1 = top_feats.shape[0]
n2 = random_feats.shape[0]

uw_net = np.zeros((n1, n1))
for (i, j) in it.combinations(range(n1), 2):
    if np.sign(top_weights[i]) == np.sign(top_weights[j]):
        uw_net[i, j] = 1.0
        uw_net[j, i] = 1.0
    else:
        uw_net[i, j] = -1.0
        uw_net[j, i] = -1.0
adj_uw = np.block([
            [uw_net, np.zeros((n1, n2))],
            [np.zeros((n2, n1)), np.eye(n2, n2)]
])

# weighted network with weights in [-1, 1]
w_net = np.zeros((n1, n1))
# linearly rescale regression coefficients to [-1, 1]
rescaled_weights = 2 * ((top_weights - top_weights.min())/(top_weights.max() - top_weights.min())) - 1
# then just take 1 - the difference between weights for each feature pair,
# this will also be between -1 and 1
for (i, j) in it.combinations(range(n1), 2):
    w_diff = np.abs(rescaled_weights[i] - rescaled_weights[j])
    w_net[i, j] = w_diff
    w_net[j, i] = w_diff
adj_w = np.block([
            [w_net, np.zeros((n1, n2))],
            [np.zeros((n2, n1)), np.eye(n2)]
])

# dense network with random edges
adj_random = np.random.uniform(low=-1.0, high=1.0, size=(n1+n2, n1+n2))
np.fill_diagonal(adj_random, 0)

nodelist = np.concatenate((top_feats, random_feats))

In [29]:
tcga_subset = tcga_train_df.loc[:, nodelist]
tcga_subset.head()

,1643,64782,51065,7508,4193,1026,9526,581,2232,23612,...,6611,5409,5266,1534,949,57062,7095,57198,56944,6745
sample_id,,,,,,,,,,,,,,,,,,,,,
TCGA-LL-A73Z-01,396.0,509.0,1690.0,1280.0,1090.0,1130.0,1210.0,1030.0,260.0,1690.0,...,1770.0,11.900,112.000,2530.0,885.0,2960.0,2680.0,742.0,1850.0,2810.0
TCGA-55-8207-01,360.0,789.0,1070.0,1360.0,1610.0,4470.0,1130.0,739.0,321.0,2050.0,...,792.0,59.600,2.180,2960.0,355.0,4500.0,6330.0,1280.0,2700.0,4250.0
TCGA-FF-A7CR-01,571.0,1670.0,965.0,1100.0,2500.0,1970.0,1170.0,1840.0,347.0,76.2,...,1660.0,0.385,0.385,299.0,1130.0,3330.0,2090.0,1010.0,162.0,3080.0
TCGA-BK-A13C-11,409.0,310.0,1060.0,1200.0,1190.0,801.0,461.0,391.0,304.0,863.0,...,1010.0,17.100,3.690,641.0,1390.0,3850.0,4970.0,2980.0,5920.0,4080.0
TCGA-EB-A6L9-06,394.0,1200.0,614.0,722.0,244.0,676.0,1660.0,1250.0,393.0,693.0,...,1930.0,1.420,1.420,591.0,5630.0,4340.0,790.0,4390.0,424.0,2980.0


In [31]:
def save_numpy_to_el(adj, nodelist, filename):
    G = nx.from_numpy_matrix(adj)
    G = nx.relabel_nodes(G, {ix: n for ix, n in enumerate(nodelist)})
    nx.write_weighted_edgelist(G, filename, delimiter='\t')
    
if not os.path.exists(cfg.networks_dir):
    os.makedirs(cfg.networks_dir)
    
tcga_subset.to_csv(os.path.join(cfg.data_dir, 'tcga_train_sim_subset_p{}.tsv'.format(n_weights)), 
                   float_format='%.4f', sep='\t')
save_numpy_to_el(adj_uw, nodelist, os.path.join(cfg.networks_dir,
                                                'tcga_top_genes_uw_p{}.tsv'.format(n_weights)))
save_numpy_to_el(adj_w, nodelist, os.path.join(cfg.networks_dir,
                                               'tcga_top_genes_w_p{}.tsv'.format(n_weights)))
save_numpy_to_el(adj_random, nodelist, os.path.join(cfg.networks_dir,
                                                    'tcga_top_genes_random_p{}.tsv'.format(n_weights)))
np.savetxt(os.path.join(cfg.networks_dir, 'tcga_top_genes_nodelist.tsv'.format(n_weights)),
           nodelist, fmt='%s', delimiter='\t')

In [41]:
nodelist

array(['1643', '64782', '51065', '7508', '4193', '1026', '9526', '581',
       '2232', '23612', '79689', '8837', '85461', '1244', '360', '27286',
       '26225', '51026', '6374', '2280'], dtype=object)